# Word Embeddings: Ungraded Practice Notebook

In this ungraded notebook, you'll try out all the individual techniques that you learned about in the lecture. Practicing on small examples will prepare you for the graded assignment, where you will combine the techniques in more advanced ways to create word embeddings from a real-life corpus.

This notebook is made of two main parts: data preparation, and the continuous bag-of-words (CBOW) model.

To get started, import and initialize all the libraries you will need.

In [1]:
import os
import pickle
import re

import emoji
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

from utils import get_dict

#nltk.download('punkt') broken ... workaround below

In [2]:
HOME = os.environ['HOME']
punkt_path = f'{HOME}/nltk_data/tokenizers/punkt/PY3/english.pickle'

In [3]:
with open(punkt_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Data preparation

In the data preparation phase, starting with a corpus of text, you will:

- Clean and tokenize the corpus.

- Extract the pairs of context words and center word that will make up the training data set for the CBOW model. The context words are the features that will be fed into the model, and the center words are the target values that the model will learn to predict.

- Create simple vector representations of the context words (features) and center words (targets) that can be used by the neural network of the CBOW model.

## Cleaning and tokenization

To demonstrate the cleaning and tokenization process, consider a corpus that contains emojis and various punctuation signs.

In [4]:
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'
print(f'Corpus:  {corpus}')
data = re.sub(r'[,!?;-]+', '.', corpus)
print(f'After cleaning punctuation:  {data}')

Corpus:  Who ❤️ "word embeddings" in 2020? I do!!!
After cleaning punctuation:  Who ❤️ "word embeddings" in 2020. I do.


In [5]:
print(f'Initial string:  {data}')
data = nltk.word_tokenize(data)
print(f'After tokenization:  {data}')

Initial string:  Who ❤️ "word embeddings" in 2020. I do.
After tokenization:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']


In [6]:
print(f'Initial list of tokens:  {data}')
data = [ ch.lower() for ch in data
         if ch.isalpha()
         or ch == '.'
         or emoji.get_emoji_regexp().search(ch)
       ]
print(f'After cleaning:  {data}')

Initial list of tokens:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']
After cleaning:  ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.']


In [7]:
# wrap in func
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)
    data = [ch.lower() for ch in data 
            if ch.isalpha()
            or ch == '.'
            or emoji.get_emoji_regexp().search(ch)]
    return data

In [8]:
corpus = 'I am happy because I am learning'
print(f'Corpus:  {corpus}')
words = tokenize(corpus)
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


In [9]:
tokenize('A man, a plan, a canal: "Panama!"')

['a', 'man', '.', 'a', 'plan', '.', 'a', 'canal', 'panama', '.']

## Sliding window of words
Now that you have transformed the corpus into a list of clean tokens, you can slide a window of words across this list. For each window you can extract a center word and the context words.

The `get_windows` function in the next cell was introduced in the lecture.

In [10]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i + 1):(i + C + 1)]
        yield context_words, center_word
        i += 1

The first argument of this function is a list of words (or tokens). The second argument, `C`, is the context half-size. Recall that for a given center word, the context words are made of `C` words to the left and `C` words to the right of the center word.

Here is how you can use this function to extract context words and center words from a list of tokens. These context and center words will make up the training set that you will use to train the CBOW model.

In [11]:
for x, y in get_windows(
    ['i', 'am', 'happy', 'because', 'i', 'am', 'learning'], 2):
    print(f'{x}\t{y}')

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


The first example of the training set is made of:

- the context words "i", "am", "because", "i",

- and the center word to be predicted: "happy".

**Now try it out yourself. In the next cell, you can change both the sentence and the context half-size.**

In [12]:
for x, y in get_windows(
    tokenize(
        "I'm a lumberjack and I'm ok, I sleep all night and I work all day"), 
    3):
    print(f'{x}\t{y}')

['i', 'a', 'lumberjack', 'i', 'ok', '.']	and
['a', 'lumberjack', 'and', 'ok', '.', 'i']	i
['lumberjack', 'and', 'i', '.', 'i', 'sleep']	ok
['and', 'i', 'ok', 'i', 'sleep', 'all']	.
['i', 'ok', '.', 'sleep', 'all', 'night']	i
['ok', '.', 'i', 'all', 'night', 'and']	sleep
['.', 'i', 'sleep', 'night', 'and', 'i']	all
['i', 'sleep', 'all', 'and', 'i', 'work']	night
['sleep', 'all', 'night', 'i', 'work', 'all']	and
['all', 'night', 'and', 'work', 'all', 'day']	i


## Transforming words into vectors for the training set
To finish preparing the training set, you need to transform the context words and center words into vectors.

### Mapping words to indices and indices to words

The center words will be represented as one-hot vectors, and the vectors that represent context words are also based on one-hot vectors.

To create one-hot word vectors, you can start by mapping each unique word to a unique integer (or index). We have provided a helper function, `get_dict`, that creates a Python dictionary that maps words to integers and back.

In [13]:
word2ind, ind2word = get_dict(words)

In [14]:
word2ind

{'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4}

In [15]:
print("Index of the word 'i':  ", word2ind['i'])

Index of the word 'i':   3


In [16]:
ind2word

{0: 'am', 1: 'because', 2: 'happy', 3: 'i', 4: 'learning'}

In [17]:
print("Word which has index 2:", ind2word[2] )

Word which has index 2: happy


In [18]:
V = len(word2ind)
print("Size of vocabulary:", V)

Size of vocabulary: 5


### Getting one-hot word vectors

Recall from the lecture that you can easily convert an integer, $n$, into a one-hot vector.

Consider the word "happy". First, retrieve its numeric index.

In [19]:
n = word2ind['happy']
n

2

In [20]:
center_word_vector = np.zeros(V)
center_word_vector

array([0., 0., 0., 0., 0.])

In [21]:
len(center_word_vector) == V

True

In [22]:
center_word_vector[n] = 1
center_word_vector

array([0., 0., 1., 0., 0.])

In [23]:
def word_to_one_hot_vector(word, word2ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2ind[word]] = 1
    return one_hot_vector

In [24]:
word_to_one_hot_vector('happy', word2ind, V)

array([0., 0., 1., 0., 0.])

In [25]:
word_to_one_hot_vector('learning', word2ind, V)

array([0., 0., 0., 0., 1.])

### Getting context word vectors
To create the vectors that represent context words, you will calculate the average of the one-hot vectors representing the individual words.

Let's start with a list of context words.

In [26]:
context_words = ['i', 'am', 'because', 'i']

Using Python's list comprehension construct and the `word_to_one_hot_vector` function that you created in the previous section, you can create a list of one-hot vectors representing each of the context words.

In [27]:
context_words_vectors = [word_to_one_hot_vector(w, word2ind, V) 
                         for w in context_words]
context_words_vectors

[array([0., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0.])]

And you can now simply get the average of these vectors using numpy's `mean` function, to get the vector representation of the context words.

In [28]:
np.mean(context_words_vectors, axis=0)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

Note the `axis=0` parameter that tells `mean` to calculate the average of the rows (if you had wanted the average of the columns, you would have used `axis=1`).

**Now create the `context_words_to_vector` function that takes in a list of context words, a word-to-index dictionary, and a vocabulary size, and outputs the vector representation of the context words.**

In [32]:
def context_words_to_vector(context_words, word2ind, V):
    context_word_vectors = [word_to_one_hot_vector(w, word2ind, V) 
                            for w in context_words]
    context_word_vectors = np.mean(context_word_vectors, axis=0)
    return context_word_vectors

In [33]:
context_words_to_vector(['i', 'am', 'because', 'i'], word2ind, V)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

## Building the training set
You can now combine the functions that you created in the previous sections, to build a training set for the CBOW model, starting from the following tokenized corpus.

In [34]:
words

['i', 'am', 'happy', 'because', 'i', 'am', 'learning']

To do this you need to use the sliding window function (`get_windows`) to extract the context words and center words, and you then convert these sets of words into a basic vector representation using `word_to_one_hot_vector` and `context_words_to_vector`.

In [36]:
# reminder: 2 is the context half-size
for context_words, center_word in get_windows(words, 2):
    print(f'Context words:  {context_words} -> '
          f'{context_words_to_vector(context_words, word2ind, V)}')
    print(f'Center word:  {center_word} -> '
          f'{word_to_one_hot_vector(center_word, word2ind, V)}')
    print()

Context words:  ['i', 'am', 'because', 'i'] -> [0.25 0.25 0.   0.5  0.  ]
Center word:  happy -> [0. 0. 1. 0. 0.]

Context words:  ['am', 'happy', 'i', 'am'] -> [0.5  0.   0.25 0.25 0.  ]
Center word:  because -> [0. 1. 0. 0. 0.]

Context words:  ['happy', 'because', 'am', 'learning'] -> [0.25 0.25 0.25 0.   0.25]
Center word:  i -> [0. 0. 0. 1. 0.]



In this practice notebook you'll be performing a single iteration of training using a single example, but in this week's assignment you'll train the CBOW model using several iterations and batches of example.
Here is how you would use a Python generator function (remember the `yield` keyword from the lecture?) to make it easier to iterate over a set of examples.

In [37]:
def get_training_example(words, C, word2ind, V):
    for ctx_words, center_word in get_windows(words, C):
        yield (context_words_to_vector(ctx_words, word2ind, V), 
               word_to_one_hot_vector(center_word, word2ind, V))

The output of this function can be iterated on to get successive context word vectors and center word vectors, as demonstrated in the next cell.

In [39]:
for context_words_vector, center_word_vector in get_training_example(
        words, 2, word2ind, V):
    print(f'Context words vector: {context_words_vector}')
    print(f'Center word vector: {center_word_vector}')
    print()

Context words vector: [0.25 0.25 0.   0.5  0.  ]
Center word vector: [0. 0. 1. 0. 0.]

Context words vector: [0.5  0.   0.25 0.25 0.  ]
Center word vector: [0. 1. 0. 0. 0.]

Context words vector: [0.25 0.25 0.25 0.   0.25]
Center word vector: [0. 0. 0. 1. 0.]



Your training set is ready, you can now move on to the CBOW model itself.

# The continuous bag-of-words model
The CBOW model is based on a neural network, the architecture of which looks like the figure below, as you'll recall from the lecture.

This part of the notebook will walk you through:
- The two activation functions used in the neural network.
- Forward propagation.
- Cross-entropy loss.
- Backpropagation.
- Gradient descent.
- Extracting the word embedding vectors from the weight matrices once the neural network has been trained.

## Activation functions
Let's start by implementing the activation functions, ReLU and softmax.

### ReLU
ReLU is used to calculate the values of the hidden layer, in the following formulas:

\begin{align}
 \mathbf{z_1} &= \mathbf{W_1}\mathbf{x} + \mathbf{b_1}  \tag{1} \\
 \mathbf{h} &= \mathrm{ReLU}(\mathbf{z_1})  \tag{2} \\
\end{align}

In [40]:
np.random.seed(10)
z_1 = 10*np.random.rand(5, 1) - 5
z_1

array([[ 2.71320643],
       [-4.79248051],
       [ 1.33648235],
       [ 2.48803883],
       [-0.01492988]])

In [41]:
h = z_1.copy()
h[h < 0] = 0
h

array([[2.71320643],
       [0.        ],
       [1.33648235],
       [2.48803883],
       [0.        ]])

In [42]:
def relu(z):
    result = z.copy()
    result[result < 0] = 0
    return result

In [43]:
z = np.array([[-1.25459881], 
              [ 4.50714306], 
              [ 2.31993942], 
              [ 0.98658484], 
              [-3.4398136 ]])
relu(z)

array([[0.        ],
       [4.50714306],
       [2.31993942],
       [0.98658484],
       [0.        ]])

### Softmax
The second activation function that you need is softmax. This function is used to calculate the values of the output layer of the neural network, using the following formulas:

\begin{align}
 \mathbf{z_2} &= \mathbf{W_2}\mathbf{h} + \mathbf{b_2}   \tag{3} \\
 \mathbf{\hat y} &= \mathrm{softmax}(\mathbf{z_2})   \tag{4} \\
\end{align}

To calculate softmax of a vector $\mathbf{z}$, the $i$-th component of the resulting vector is given by:

$$ \textrm{softmax}(\textbf{z})_i = \frac{e^{z_i} }{\sum\limits_{j=1}^{V} e^{z_j} }  \tag{5} $$

Let's work through an example.

In [44]:
z = np.array([9, 8, 11, 10, 8.5])
ez = np.exp(z)
sum_ez = ez.sum()
ez / sum_ez

array([0.08276948, 0.03044919, 0.61158833, 0.22499077, 0.05020223])

In [47]:
def softmax(z):
    ez = np.exp(z)
    return ez / ez.sum()

In [48]:
softmax(z)

array([0.08276948, 0.03044919, 0.61158833, 0.22499077, 0.05020223])

## Dimensions: 1-D arrays vs 2-D column vectors

Before moving on to implement forward propagation, backpropagation, and gradient descent, let's have a look at the dimensions of the vectors you've been handling until now.

Create a vector of length $V$ filled with zeros.

In [50]:
x_array = np.zeros(V)
x_array, x_array.shape

(array([0., 0., 0., 0., 0.]), (5,))

In [51]:
x_colvec = x_array.copy().reshape(V, 1)
x_colvec

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [52]:
x_colvec.shape

(5, 1)

So you now have a 5x1 matrix that you can use to perform standard matrix multiplication.

## Forward propagation
Let's dive into the neural network itself, which is shown below with all the dimensions and formulas you'll need.

Set $N$ equal to 3. Remember that $N$ is a hyperparameter of the CBOW model that represents the size of the word embedding vectors, as well as the size of the hidden layer.

In [53]:
N = 3

### Initialization of the weights and biases
Before you start training the neural network, you need to initialize the weight matrices and bias vectors with random values.

In the assignment you will implement a function to do this yourself using `numpy.random.rand`. In this notebook, we've pre-populated these matrices and vectors for you.

In [54]:
W1 = np.array([
    [ 0.41687358,  0.08854191, -0.23495225,  0.28320538,  0.41800106],
    [ 0.32735501,  0.22795148, -0.23951958,  0.4117634 , -0.23924344],
    [ 0.26637602, -0.23846886, -0.37770863, -0.11399446,  0.34008124]])

W2 = np.array([[-0.22182064, -0.43008631,  0.13310965],
               [ 0.08476603,  0.08123194,  0.1772054 ],
               [ 0.1871551 , -0.06107263, -0.1790735 ],
               [ 0.07055222, -0.02015138,  0.36107434],
               [ 0.33480474, -0.39423389, -0.43959196]])

b1 = np.array([[ 0.09688219],
               [ 0.29239497],
               [-0.27364426]])

b2 = np.array([[ 0.0352008 ],
               [-0.36393384],
               [-0.12775555],
               [-0.34802326],
               [-0.07017815]])

In [55]:
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of W1: {W1.shape} (NxV)')
print(f'size of b1: {b1.shape} (Nx1)')
print(f'size of W2: {W1.shape} (VxN)')
print(f'size of b2: {b2.shape} (Vx1)')

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of W1: (3, 5) (NxV)
size of b1: (3, 1) (Nx1)
size of W2: (3, 5) (VxN)
size of b2: (5, 1) (Vx1)


### Training example
Run the next cells to get the first training example, made of the vector representing the context words "i am because i", and the target which is the one-hot vector representing the center word "happy".

> You don't need to worry about the Python syntax, but there are some explanations below if you want to know what's happening behind the scenes.

In [57]:
training_examples = get_training_example(words, 2, word2ind, V)

> `get_training_examples`, which uses the `yield` keyword, is known as a generator. When run, it builds an iterator, which is a special type of object that... you can iterate on (using a `for` loop for instance), to retrieve the successive values that the function generates.
>
> In this case `get_training_examples` `yield`s training examples, and iterating on `training_examples` will return the successive training examples.

In [58]:
x_array, y_array = next(training_examples)

> `next` is another special keyword, which gets the next available value from an iterator. Here, you'll get the very first value, which is the first training example. If you run this cell again, you'll get the next value, and so on until the iterator runs out of values to return.
>
> In this notebook `next` is used because you will only be performing one iteration of training. In this week's assignment with the full training over several iterations you'll use regular `for` loops with the iterator that supplies the training examples.

The vector representing the context words, which will be fed into the neural network, is:

In [59]:
x_array, y_array

(array([0.25, 0.25, 0.  , 0.5 , 0.  ]), array([0., 0., 1., 0., 0.]))

Now convert these vectors into matrices (or 2D arrays) to be able to perform matrix multiplication on the right types of objects, as explained above.

In [60]:
x = x_array.copy()
x.shape = (V, 1)
print('x')
print(x)
print()

y = y_array.copy()
y.shape = (V, 1)
print('y')
print(y)

x
[[0.25]
 [0.25]
 [0.  ]
 [0.5 ]
 [0.  ]]

y
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


### Values of the hidden layer

Now that you have initialized all the variables that you need for forward propagation, you can calculate the values of the hidden layer using the following formulas:

\begin{align}
 \mathbf{z_1} = \mathbf{W_1}\mathbf{x} + \mathbf{b_1}  \tag{1} \\
 \mathbf{h} = \mathrm{ReLU}(\mathbf{z_1})  \tag{2} \\
\end{align}

First, you can calculate the value of $\mathbf{z_1}$.

In [61]:
z1 = np.dot(W1, x) + b1
print(z1)

h = relu(z1)
h

[[ 0.36483875]
 [ 0.63710329]
 [-0.3236647 ]]


array([[0.36483875],
       [0.63710329],
       [0.        ]])

### Values of the output layer

Here are the formulas you need to calculate the values of the output layer, represented by the vector $\mathbf{\hat y}$:

\begin{align}
 \mathbf{z_2} &= \mathbf{W_2}\mathbf{h} + \mathbf{b_2}   \tag{3} \\
 \mathbf{\hat y} &= \mathrm{softmax}(\mathbf{z_2})   \tag{4} \\
\end{align}

**First, calculate $\mathbf{z_2}$.**

In [63]:
z2 = np.dot(W2, h) + b2
print(z2)

y_hat = softmax(z2)
y_hat

[[-0.31973737]
 [-0.28125477]
 [-0.09838369]
 [-0.33512159]
 [-0.19919612]]


array([[0.18519074],
       [0.19245626],
       [0.23107446],
       [0.18236353],
       [0.20891502]])

In [65]:
# prediction:
print(ind2word[np.argmax(y_hat)])

happy


Well done, you've completed the forward propagation phase!

## Cross-entropy loss

Now that you have the network's prediction, you can calculate the cross-entropy loss to determine how accurate the prediction was compared to the actual target.

> Remember that you are working on a single training example, not on a batch of examples, which is why you are using *loss* and not *cost*, which is the generalized form of loss.

First let's recall what the prediction was.